In [137]:
import os
import pandas as pd
import numpy as np

import plotly.graph_objects as go
import plotly.express as px

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

from imblearn.over_sampling import RandomOverSampler

from scipy.io import savemat, loadmat

In [138]:
dataset = pd.read_csv('/home/simone/PycharmProjects/cPDS/data/framingham.csv')

In [139]:
dataset

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4235,0,48,2.0,1,20.0,NaN,0,0,0,248.0,131.0,72.0,22.00,84.0,86.0,0
4236,0,44,1.0,1,15.0,0.0,0,0,0,210.0,126.5,87.0,19.16,86.0,NaN,0
4237,0,52,2.0,0,0.0,0.0,0,0,0,269.0,133.5,83.0,21.47,80.0,107.0,0
4238,1,40,3.0,0,0.0,0.0,0,1,0,185.0,141.0,98.0,25.60,67.0,72.0,0


In [140]:
dataset.dropna(inplace=True)

In [141]:
# correlation
fig = px.imshow(dataset.corr())
fig.show()

### Numerical Features

In [142]:
dataset_num = dataset[['glucose', 'cigsPerDay', 'diaBP', 'age', 'BMI', 'sysBP', 'heartRate', 'totChol', 'TenYearCHD']]

In [143]:
fig = px.scatter_matrix(dataset_num,
    dimensions=dataset_num.columns,
    color="TenYearCHD")
fig.update_traces(diagonal_visible=False)
fig.update_layout(autosize=False, width=1600, height=1080)
fig.show()

### Actually:

In [144]:
dataset = dataset[
        ['sysBP', 'glucose', 'age', 'totChol', 'cigsPerDay', 'diaBP', 'prevalentHyp', 'diabetes', 'BPMeds', 'male',
         'TenYearCHD']]

y = dataset['TenYearCHD']
x = dataset.drop(['TenYearCHD'], axis=1)

scaler = StandardScaler()
x = scaler.fit_transform(x)

pca = PCA(n_components=2)
x = pca.fit_transform(x)

In [145]:
xtrain, xtest, ytrain, ytest = train_test_split(x, y.to_numpy().astype(float))

In [146]:
classes = np.unique(ytrain)

fig = go.Figure()
fig.add_trace(
    go.Scatter(x=xtrain[ytrain == classes[0], 0], y=xtrain[ytrain == classes[0], 1], mode='markers',
               name='training class 0'))
fig.add_trace(
    go.Scatter(x=xtrain[ytrain == classes[1], 0], y=xtrain[ytrain == classes[1], 1], mode='markers',
               name='training class 1'))
fig.add_trace(go.Scatter(x=xtest[ytest == classes[0], 0], y=xtest[ytest == classes[0], 1], mode='markers',
                         name='test class 0'))
fig.add_trace(go.Scatter(x=xtest[ytest == classes[1], 0], y=xtest[ytest == classes[1], 1], mode='markers',
                         name='test class 1'))
fig.update_layout(title='Dataset')
fig.show()

### Test:
I train the cPDS and i test it obtaining an auc of 0.68, but the plot is bad.

<img src="newplot.png" alt="Drawing" style="width: 1000px;"/>

In [147]:
#savemat('framingham.mat',
#            {'xtrain': xtrain, 'xtest': xtest, 'ytrain': ytrain.reshape(-1, 1), 'ytest': ytest.reshape(-1, 1)})